In [2]:
import os
import pandas as pd

In [4]:
# 結合したいCSVファイルが存在するディレクトリを指定
directory = './automation_med_best_result/'

# 結合したファイルを保存するためのファイル名
output_file = 'automation_median_combined_reviews.csv'

# "re"で始まり"review.csv"で終わるファイルをリストに格納
csv_files = [f for f in os.listdir(directory) if f.startswith('re') and f.endswith('review.csv')]

# 空のデータフレームを作成
combined_df = pd.DataFrame()

# ファイルを順に読み込み、データフレームに結合
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# 'トータル所要時間'が675を超えるデータを削除
combined_df = combined_df[combined_df['トータル所要時間'] <= 675]

# 結合結果をCSVファイルとして保存
combined_df.to_csv(os.path.join(directory, output_file), index=False)

print(f"結合されたCSVファイルは {output_file} に保存されました。")


結合されたCSVファイルは automation_median_combined_reviews.csv に保存されました。


In [7]:
import os
import pandas as pd

# 複数のフォルダパスをリストで指定
directories = [
    './automation_best_result_csv/',
    './automation_med_best_result/',
    './best_result_csv/',
    # 必要に応じてフォルダパスを追加
]

# 出力先のファイル名を指定
output_file = 'combined_data.csv'

# 空のデータフレームを作成
combined_df = pd.DataFrame()

# 指定されたすべてのフォルダを走査
for directory in directories:
    # サブディレクトリも含めてCSVファイルを検索
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                
                # '日付'列が空欄の場合に'median'を入力
                df['日付'].fillna('median', inplace=True)
                
                # 'lambda'列を削除
                if 'lambda' in df.columns:
                    df.drop(columns=['lambda'], inplace=True)
                
                # 'トータル所要時間'が675以下のデータのみを抽出
                df_filtered = df[df['トータル所要時間'] <= 675]
                
                # フィルタリングされたデータを結合
                combined_df = pd.concat([combined_df, df_filtered], ignore_index=True)

# データをソート（例：'日付'列でソート）
combined_df.sort_values(by='日付', ascending=True, inplace=True)

# 結合したデータを1つのCSVファイルとして保存
combined_df.to_csv(output_file, index=False)

print(f"ソートされたCSVファイルは {output_file} に保存されました。")


ソートされたCSVファイルは combined_data.csv に保存されました。


In [9]:
import pandas as pd
import ast

# popularityリスト（各アトラクションの人気度）
popularity = [476, 465, 473, 472, 480, 452, 461, 478]

# CSVファイルを読み込み
df = pd.read_csv('combined_data.csv')

# '最適プラン'の総和を計算し、'トータル満足度'に反映
for index, row in df.iterrows():
    # '最適プラン'をリストに変換（文字列をリストに変換）
    try:
        optimal_plan = ast.literal_eval(row['最適プラン'])
    except (ValueError, SyntaxError):
        print(f"エラー: 行 {index} の '最適プラン' を解析できません")
        continue
    
    # 人気度の総和を計算
    total_popularity = sum(popularity[i] for i in optimal_plan)
    
    # トータル満足度に反映
    df.at[index, 'トータル満足度'] = total_popularity

# 結果を確認
#print(df)

# 必要に応じてCSVに保存
df.to_csv('updated_combined_data.csv', index=False)


               日付  トータル満足度  トータル所要時間  \
0      2022-07-01   6521.0     639.0   
1      2022-07-01   7547.0     633.0   
2      2022-07-01   7525.0     656.0   
3      2022-07-01   7050.0     600.0   
4      2022-07-01   7061.0     609.0   
...           ...      ...       ...   
10245      median   6138.0     675.0   
10246      median   5646.0     673.0   
10247      median   6028.0     671.5   
10248      median   5153.0     653.5   
10249      median   6054.0     661.0   

                                                   予定プラン  \
0      [5, 6, 1, 6, 2, 3, 4, 4, 4, 4, 4, 3, 5, 3, 3, ...   
1      [7, 3, 4, 2, 3, 3, 5, 4, 2, 4, 2, 5, 3, 4, 4, ...   
2      [4, 4, 3, 5, 2, 5, 3, 2, 4, 2, 4, 4, 2, 4, 3, ...   
3          [6, 1, 1, 4, 0, 3, 0, 1, 5, 4, 0, 1, 3, 1, 4]   
4          [2, 3, 0, 3, 0, 4, 0, 0, 5, 0, 3, 1, 1, 1, 1]   
...                                                  ...   
10245      [5, 6, 3, 4, 7, 4, 7, 7, 4, 5, 4, 2, 3, 4, 4]   
10246   [7, 6, 6, 2, 4, 3, 4, 2, 4, 5, 